### Multiplicação de matrizes com numpy


In [1]:
import pandas as pd
from pathlib import Path

INPUT_PATH = Path('dados_tratados_completo.csv')
OUTPUT_PATH = Path('dados_tratados.csv')

if not INPUT_PATH.exists():
    raise FileNotFoundError('dados_tratados_completo.csv nao encontrado na raiz do projeto')

def read_csv_with_fallback(path):
    for enc in ('utf-8', 'cp1252', 'latin1'):
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False, dtype=str)
        except UnicodeDecodeError:
            continue
    return pd.read_csv(path, low_memory=False, dtype=str)

df = read_csv_with_fallback(INPUT_PATH)
df.head()


,uf,mes_referencia,idade,sexo,escolaridade,teve_febre,teve_tosse,teve_dor_garganta,teve_dificuldade_respirar,teve_dor_peito,perda_olfato_paladar,procurou_saude,resultado_exame,medida_isolamento,trabalhou_semana_passada,afastado_trabalho,tipo_trabalho,faixa_rendimento,trabalho_remoto,auxilio_emergencial
0,Rondônia,07,035,Homem,Médio completo,Não,Não,Não,Não,Não,Não,Não aplicável,Não aplicável,"Reduziu o contato com as pessoas, mas continuo...",Sim,Não aplicável,Outro técnico ou profissional de nível médio,801 - 1.600,Não aplicável,Sim
1,Rondônia,07,029,Mulher,Superior completo,Não,Não,Não,Não,Não,Não,Não aplicável,Não aplicável,Ficou em casa e só saiu em caso de necessidade...,Não,Sim,não aplicável,Não aplicável,Não aplicável,Sim
2,Rondônia,07,013,Homem,Fundamental incompleto,Não,Não,Não,Não,Não,Não,Não aplicável,Não aplicável,Ficou rigorosamente em casa,Não aplicável,Não aplicável,não aplicável,Não aplicável,Não aplicável,Sim
3,Rondônia,07,010,Homem,Fundamental incompleto,Não,Não,Não,Não,Não,Não,Não aplicável,Não aplicável,Ficou rigorosamente em casa,Não aplicável,Não aplicável,não aplicável,Não aplicável,Não aplicável,Sim
4,Rondônia,07,057,Mulher,Fundamental incompleto,Não,Não,Não,Não,Não,Não,Não aplicável,Não aplicável,Ficou em casa e só saiu em caso de necessidade...,Não,Não,não aplicável,Não aplicável,Não aplicável,Não


In [2]:
EXPECTED_COLS = [
    'uf',
    'mes_referencia',
    'idade',
    'sexo',
    'escolaridade',
    'teve_febre',
    'teve_tosse',
    'teve_dor_garganta',
    'teve_dificuldade_respirar',
    'teve_dor_peito',
    'perda_olfato_paladar',
    'procurou_saude',
    'resultado_exame',
    'medida_isolamento',
    'trabalhou_semana_passada',
    'afastado_trabalho',
    'tipo_trabalho',
    'faixa_rendimento',
    'trabalho_remoto',
    'auxilio_emergencial',
]

obj_cols = df.select_dtypes(include='object').columns
df[obj_cols] = df[obj_cols].apply(lambda s: s.str.strip())

df['mes_referencia'] = df['mes_referencia'].astype(str).str.zfill(2)
df['idade'] = pd.to_numeric(df['idade'], errors='coerce').astype('Int64')

missing_cols = [c for c in EXPECTED_COLS if c not in df.columns]
for col in missing_cols:
    df[col] = pd.NA

df = df[EXPECTED_COLS]
missing_cols


[]

In [3]:
df.to_csv(OUTPUT_PATH, index=False, encoding='utf-8')
df['mes_referencia'].value_counts().head()


mes_referencia
09    387298
08    386520
07    384166
Name: count, dtype: int64